In [1]:
from __future__ import print_function
import madmom as mmm
from miran import *
from subprocess import call
from time import clock

In [2]:
# ANALYSIS PATHS
sa_path = '/Users/angel/Git/miran/scripts/sonic-annotator'
ff = folderfiles('/Users/angel/Desktop/giantsteps/', ext='.mp3')

idx = 0
beat_tracker = mmm.features.beats.DBNDownBeatTrackingProcessor(beats_per_bar=16, fps=100, downbeats=True)

for f in ff:
    if any(soundfile_type in f for soundfile_type in AUDIO_FILE_EXTENSIONS):
        
        clock()
        print("Analysing {}".format(f), end='')
        
        # TEMPORARILY RENAME FILE TO AVOID ILEGAL CHARACTERS.
        fname, fext = os.path.splitext(f)
        fdir, fname = os.path.split(fname)
        fname = os.path.join(fdir, str(idx))
        f2 = fname + fext
        os.rename(f, f2)

        # INIT PLACEHOLDERS
        output_df = pd.DataFrame()
        #harmonicity_means = [np.nan]
        #harmonicity_std   = [np.nan]
        key1              = [np.nan]
        corr1             = [np.nan]
        key2              = [np.nan]
        corr2             = [np.nan]
        confidence        = [np.nan]

        # DOWNBEAT COMPUTATION
        beat_activations = mmm.features.beats.RNNDownBeatProcessor()(f2)
        downbeat_positions = beat_tracker(beat_activations)
        
        # VAMP-PLUGIN ANALYSIS
        call('{0}/sonic-annotator -r -t {0}/nnls.n3 -w csv --csv-force "{1}"'.format(sa_path, f2), shell=True)
        #harmonicity = pd.read_csv(os.path.splitext(f2)[0] + '_vamp_vamp-libxtract_spectral_inharmonicity_spectral_inharmonicity.csv', header=None)
        nnls = pd.read_csv(os.path.splitext(f2)[0] + '_vamp_nnls-chroma_nnls-chroma_chroma.csv', header=None)
        
        # ADD ZERO POSTION
        if 0 not in downbeat_positions:
            downbeat_positions = np.append(downbeat_positions, 0.00)
            downbeat_positions = np.roll(downbeat_positions, 1)

        # MAKE CALCULATION BASED ON HYPERMETRICAL DIVISIONS
        for position in range(len(downbeat_positions) - 1):
            #temp = harmonicity[harmonicity[0] >= downbeat_positions[position]]
            #temp = harmonicity[harmonicity[0] <  downbeat_positions[position + 1]]
            #harmonicity_means.append(temp.mean()[1])
            #harmonicity_std.append(temp.std()[1])
            temp = nnls[nnls[0] >= downbeat_positions[position]]
            temp = nnls[nnls[0] <  downbeat_positions[position + 1]]
            key_estimate = estimate_key(np.roll(temp.drop(0,1).mean(), -3), 
                                        profile_type='bgate',
                                        interpolation='linear',
                                        candidates=2,
                                        conf_thres=0,
                                        vocabulary=2)
            key1.append(key_estimate[1][0])
            key2.append(key_estimate[1][1])
            corr1.append(key_estimate[2][0])
            corr2.append(key_estimate[2][1])
            confidence.append(key_estimate[3])
            
        # SAVE ALL RESULTS PER HYPERMEASURE TO PD DATAFRAME.
        output_df['downbeats']  = downbeat_positions
        #output_df['harm_mu']    = np.roll(harmonicity_means, -1)
        #output_df['harm_std']   = np.roll(harmonicity_std, -1)
        output_df['key1']       = np.roll(key1, -1)
        output_df['corr1']      = np.roll(corr1, -1)
        output_df['key2']       = np.roll(key2, -1)
        output_df['corr2']      = np.roll(corr2, -1)
        output_df['confidence'] = np.roll(confidence, -1)
        
        print(" in {} seconds.".format(clock()))
        
        # REMOVE INTERMEDIATE FILES AND RENAME WITH ORIGINAL NAME
        os.remove(os.path.splitext(f2)[0] + '_vamp_nnls-chroma_nnls-chroma_chroma.csv')
        #os.remove(os.path.splitext(f2)[0] + '_vamp_vamp-libxtract_spectral_inharmonicity_spectral_inharmonicity.csv')
        os.rename(f2, f)
        idx += 1

        # MAKE GLOBAL KEY ESTIMATION BASED ON HYPERMETRICAL KEYS
        global_keys_candidates = []
        global_keys_confidence = []
        for item in output_df.key1.unique():
            if item != 'nan':
                key_df = output_df[output_df.key1 == item]
                global_keys_candidates.append(item)
                global_keys_confidence.append(key_df.confidence.sum())

        global_key = global_keys_candidates[global_keys_confidence.index(max(global_keys_confidence))]    
        
        # SAVE SINGLE GLOBAL ESTIMATION TO TEXT FILE AND REMOVE INTERMEDIATE FILES
        global_key_dir = '/Users/angel/Desktop/giantkeys'
        with open(os.path.join(global_key_dir, os.path.split(os.path.splitext(f)[0])[1] + '.txt'), 'w') as textfile:
                  textfile.write(global_key)



Analysing /Users/angel/Desktop/giantsteps/3271514 Mario & Vidis - Suspend The Feeling feat. Giedre (Original Mix).mp3

/usr/local/lib/python2.7/site-packages/madmom/audio/spectrogram.py:931: RuntimeWarning: invalid value encountered in maximum
  np.maximum(diff, 0, out=diff)


 in 31.173184 seconds.
Analysing /Users/angel/Desktop/giantsteps/4049579 Jabba, Fashion Viktims - Canibal Royal (Original Mix).mp3 in 58.845453 seconds.
Analysing /Users/angel/Desktop/giantsteps/3095708 Loom In Essence - Xenobotanics (Original Mix)*.mp3 in 86.577796 seconds.
Analysing /Users/angel/Desktop/giantsteps/3354212 Young Rebels, Francesco Diaz - Timbuktu (Original Mix).mp3 in 113.992357 seconds.
Analysing /Users/angel/Desktop/giantsteps/1991458 Henry Saiz - 1981 (Original Mix).mp3 in 141.618034 seconds.
Analysing /Users/angel/Desktop/giantsteps/1575015 Rasmus Faber, Emily Mcewan - Never Figure Out (Stimming Remix).mp3 in 169.275299 seconds.
Analysing /Users/angel/Desktop/giantsteps/5005412 Oliver Heldens - Gecko (Original Mix)*.mp3 in 196.698887 seconds.
Analysing /Users/angel/Desktop/giantsteps/4297635 V I F - A Touching Tune (Original Mix).mp3 in 224.329411 seconds.
Analysing /Users/angel/Desktop/giantsteps/2448121 deadmau5 - Where My Keys (Original Mix).mp3 in 251.762059 se

Analysing /Users/angel/Desktop/giantsteps/61415 Sonorous - Protonic (Ronski Speed Remix).mp3 in 1858.994872 seconds.
Analysing /Users/angel/Desktop/giantsteps/1939732 Emptyset - Altogether Lost feat. Cornelius Harris (Ripperton's Underground Kingdom Mix).mp3 in 1887.00359 seconds.
Analysing /Users/angel/Desktop/giantsteps/981591 JPLS, Ambivalent - Frontstab (Original Mix).mp3 in 1914.708827 seconds.
Analysing /Users/angel/Desktop/giantsteps/835903 Dirty South - We Are feat. Rudy (Original Mix).mp3 in 1942.339453 seconds.
Analysing /Users/angel/Desktop/giantsteps/4846007 Arthur Deep - These Barracks Look Nice (Fake Truth Remix).mp3 in 1969.948715 seconds.
Analysing /Users/angel/Desktop/giantsteps/1626779 Dima Gafner, Garry Lachman - Swing 82 (Weekend Heroes Remix).mp3 in 1997.982467 seconds.
Analysing /Users/angel/Desktop/giantsteps/2741219 Kaiserdisco - Lurin (Original Mix).mp3 in 2025.672535 seconds.
Analysing /Users/angel/Desktop/giantsteps/2070773 Cab Drivers - Get It (Original Mix)

Analysing /Users/angel/Desktop/giantsteps/4453556 Snook - Indigo (Original Mix).mp3 in 3664.931725 seconds.
Analysing /Users/angel/Desktop/giantsteps/3518921 Jori Hulkkonen, Third Culture - Options Feat. Harri Falck (Radio Edit).mp3 in 3692.63538 seconds.
Analysing /Users/angel/Desktop/giantsteps/3938837 Finnebassen, Gundelach - Footsteps (Original Mix).mp3 in 3720.410491 seconds.
Analysing /Users/angel/Desktop/giantsteps/1611446 Neshi Futuro - Glow (Original Mix).mp3 in 3747.9398 seconds.
Analysing /Users/angel/Desktop/giantsteps/5070608 FTampa - Twice (Original Mix).mp3 in 3775.748049 seconds.
Analysing /Users/angel/Desktop/giantsteps/3568438 Blu Mar Ten - The Beginning (Joe Syntax Remix)*.mp3 in 3803.35009 seconds.
Analysing /Users/angel/Desktop/giantsteps/997308 Lindstrom, Christabelle - Baby Can't Stop (Album Version).mp3 in 3831.20582 seconds.
Analysing /Users/angel/Desktop/giantsteps/435443 Dubfire - I Feel Speed (Original Club Mix).mp3 in 3858.82037 seconds.
Analysing /Users/an

...

Analysing /Users/angel/Desktop/giantsteps/5013910 Fatboy Slim - Eat Sleep Rave Repeat (Dimitri Vegas & Like Mike & Ummet Ozcan Tomorrowland Remix)*.mp3 in 9379.935045 seconds.
Analysing /Users/angel/Desktop/giantsteps/5264038 Skrillex - All Is Fair In Love And Brostep (Original Mix).mp3 in 9409.534842 seconds.
Analysing /Users/angel/Desktop/giantsteps/1681077 Marco Carola - Black Box (Original Mix).mp3 in 9438.959973 seconds.
Analysing /Users/angel/Desktop/giantsteps/4327665 Santos - Zagros Fest (Original Mix)*.mp3 in 9468.286084 seconds.
Analysing /Users/angel/Desktop/giantsteps/2075331 Fergie DJ - Distant Stab (Silicone Soul's Darkroom Dub Mix).mp3 in 9497.437074 seconds.
Analysing /Users/angel/Desktop/giantsteps/3395666 Guy Gerber - The Mirror Game (Original Mix).mp3 in 9526.637152 seconds.
Analysing /Users/angel/Desktop/giantsteps/1017921 DJ Rap - Give It All Away (Sultan & Ned Shepard Remix).mp3 in 9555.868919 seconds.
Analysing /Users/angel/Desktop/giantsteps/4957129 Amber Jolene

Analysing /Users/angel/Desktop/giantsteps/3364661 Mansel Scheffel - Together We Rise (Original Mix).mp3 in 11282.950136 seconds.
Analysing /Users/angel/Desktop/giantsteps/4452003 Zomby - Ascension (Original Mix).mp3 in 11297.509859 seconds.
Analysing /Users/angel/Desktop/giantsteps/4125044 Carlos Pardo, Amparo Balsalobre - The Sun Wont Shine Feat. Fernanda Lebrock & Demarco (Original Mix).mp3 in 11326.803074 seconds.
Analysing /Users/angel/Desktop/giantsteps/891993 The Temper Trap - Sweet Disposition (Evil Nine Mix)*.mp3 in 11356.07621 seconds.
Analysing /Users/angel/Desktop/giantsteps/4955428 Juliet Sikora, Tube & Berger - Come On Now (Set It Off) (Original Mix).mp3 in 11385.470749 seconds.
Analysing /Users/angel/Desktop/giantsteps/3015217 Bingo Players - Rattle (Original Mix).mp3 in 11414.852469 seconds.
Analysing /Users/angel/Desktop/giantsteps/3667215 Soulero - Pussycat (Original Mix).mp3 in 11443.972818 seconds.
Analysing /Users/angel/Desktop/giantsteps/4167325 King Felix - Home F

Analysing /Users/angel/Desktop/giantsteps/4898455 Armin van Buuren, Richard Bedford - Love Never Came feat. Richard Bedford (Jorn van Deynhoven Remix).mp3 in 13145.097605 seconds.
Analysing /Users/angel/Desktop/giantsteps/1673811 Vanguard - Loving Someone Else (Original Mix)*.mp3 in 13176.123755 seconds.
Analysing /Users/angel/Desktop/giantsteps/4920936 Format:B - Magic Button (Original Mix).mp3 in 13205.382948 seconds.
Analysing /Users/angel/Desktop/giantsteps/2018991 Deepfunk - Lost In Words (Original Mix).mp3 in 13234.959682 seconds.
Analysing /Users/angel/Desktop/giantsteps/3995054 Tony B - Je T'aime (Original Mix).mp3 in 13264.122622 seconds.
Analysing /Users/angel/Desktop/giantsteps/4360488 MaxNRG - Danger Zone (Original Mix).mp3 in 13293.451151 seconds.
Analysing /Users/angel/Desktop/giantsteps/1549964 Thugfucker - Disco Gnome (Tale Of Us Remix).mp3 in 13322.608214 seconds.
Analysing /Users/angel/Desktop/giantsteps/472626 The Elements Of Life - Elements Of Life feat. Blaze (Orig

Analysing /Users/angel/Desktop/giantsteps/1982269 Carl Cox - Family Guy (Layo & Bushwacka! Deeper Remix).mp3 in 15008.58367 seconds.
Analysing /Users/angel/Desktop/giantsteps/4779657 Rafijho - Paradox (Original Mix).mp3 in 15037.828836 seconds.
Analysing /Users/angel/Desktop/giantsteps/4934196 Patryk Molinari - Monkey City (Original Mix).mp3 in 15067.444231 seconds.
Analysing /Users/angel/Desktop/giantsteps/883724 Spirit Catcher, Compuphonic - Other Side Of The Game (Original Mix).mp3 in 15096.620028 seconds.
Analysing /Users/angel/Desktop/giantsteps/3935095 Tonerush - Hold Them Back (TrancEye Pres. C.R.Y. Remix).mp3 in 15125.964286 seconds.
Analysing /Users/angel/Desktop/giantsteps/4018247 Giriu Dvasios - Zemaicio Zodis (Original Mix).mp3 in 15155.449564 seconds.
Analysing /Users/angel/Desktop/giantsteps/5036476 Moby, Damien Jurado - Almost Home (Agoria Remix).mp3 in 15184.88106 seconds.
Analysing /Users/angel/Desktop/giantsteps/4062807 Funkdamentalist - The Grind (Dilby Remix).mp3 in

OSError: [Errno 2] No such file or directory